## Fundamentei

This script reads a few stocks infos available in https://fundamentei.com/ using scrapers.

In [ ]:
import os
import pandas
import sys

from tenacity import retry, wait_random, stop_after_attempt
from tqdm import tqdm

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from libs.scrapers.fundamentei import (
    StockInfosScraper
)

### Reading stock infos

In [2]:
# You can run the following javascript command to get the list direct from the https://fundamentei.com/ website
# COMMAND:
#   tickers = []; document.querySelectorAll("#__next > div > div > div > div > a > div > div > div > div.rounded-md").forEach(element => tickers.push(element.innerText)); console.log(tickers.join(","));

stock_tickers = None
stock_tickers = "PETR,ITUB,VALE,ABEV,BPAC,WEGE,BBAS,BBDC,ITSA,SANB,ELET,VIVT,SUZB,B3SA,BBSE,RDOR,RENT,SBSP,JBSS,CXSE,RADL,TIMS,RAIL,RAIZ,PRIO,CPFE,EQTL,GGBR,EGIE,CSAN,UGPA,CMIN,HAPV,CCRO,CPLE,CRFB,BRFS,VBBR,CMIG,KLBN,NTCO,ENGI,NEOE,EMBR,HYPE,CSNA,BRKM,ENEV,PSSA,ASAI,TRPL,TOTS,GMAT,EQPA,CEGR,ENMT,SMFT,MULT,LREN,MGLU,REDE,ALOS,CIEL,MDIA,CGAS,INBR,GGPS,TAEE,USIM,AURE,BPAN,GOAU,CEEB,STBP,SMTO,ALUP,VAMO,BNBR,EKTR,CYRE,MRFG,SLCE,FLRY,PORT,BRAP,POMO,SAPR,CSMG,ENAT,RRRP,IGTI,INTB,ODPV,ARZZ,SIMH,UNIP,RECV,WHRL,DXCO,ALPA,AESB,VIVA,ECOR,SRNA,CURY,SOMA,BRSR,GRND,ABCB,GOLL,YDUQ,DASA,TTEN,BAZA,ONCO,SHUL,COGN,AZUL,VULC,FRAS,MOAR,ARML,LEVE,MRVE,DIRR,BEEF,CSRN,BSLI,TUPY,LOGN,RAPT,GUAR,COCE,GPAR,EZTC,FESA,JSLG,LWSA,MOVI,MILS,IRBR,ORVR,SBFG,CBEE,CAML,CEAB,JHSF,BEES,CBAV,PLPL,HBSA,AMBP,AURA,BMEB,CLSC,MATD,GEPA,VVEO,AGRO,JALL,LOGG,ELMD,PETZ,RANI,BLAU,SOJA,MYPK,BMGB,TFCO,LIGT,ANIM,CVCB,TGMA,PNVL,KEPL,LAVV,MLAS,TASA,CSED,EVEN,DESK,BRIT,LAND,TEND,EUCA,PCAR,PGMN,SCAR,VLID,FIQE,CEBR,VSTE,BRBI,SYNE,OFSA,OPCT,PINE,FRIO,POSI,LJQQ,BMOB,DEXP,ROMI,TELB,BRIV,MDNE,TRIS,WIZC,VITT,CRPG,ZAMP,PTBL,ALPK,EMAE,BRGE,MELK,BIOM,WLMM,SEER,CSUD,AALR,APER,CLSA,AVLL,BHIA,PEAB,LIPR,RPAD,ALLD,CRIV,QUAL,AERI,PFRM,KRSA,PATI,CASH,IFCM,HBRE,MEAL,GFSA,MTSA,CGRA,MTRE,PRNR,AMER,PTNT,ETER,DOHL,AFLT,OIBR,BGIP,CAMB,HBOR,ESPA,ENJU,DMVF,HBTS,BOBR,CEDO,FHER,LPSB,RNEW,AGXY,GPIV,TECN,MSPA,NINJ,MGEL,NGRD,TKNO,LVTC,MBLY,EALT,TRAD,BAHI,JOPA,TCSA,G2DI,MERC,AZEV,PDTC,PMAM,UCAS,RDNI,BMKS,TPIS,SEQL,SHOW,RPMG,MNPR,AHEB,AMAR,CTNM,RCSL,WEST,INEP,EPAR,PLAS,CTKA,SOND,MNDL,BALM,HOOT,BMIN,VIVR,NUTR,SNSY,LUPA,DOTZ,CALI,ATMP,RSID,NORD,CTSA,DTCY,ATOM,BAUH,SGPS,JFEN,LUXM,ESTR,FIEI,RSUL,PDGR,TEKA,GSHP,HAGA,TXRX,HETA,MWET,BDLL,OSXB,NEXP,MAPT,FRTA,PSVM,PPLA".split(",")

In [3]:
# If the list is not pre defined, get stock list from other processes outputs
if not stock_tickers:
    # reading fundamentus tickers list
    fundamentus_tickers = pandas.read_csv("../outputs/fundamentus.csv", sep=";")["Papel"]
    fundamentus_tickers.columns = ['stock_ticker']

    # reading status invest tickers list
    statusinvest_tickers = pandas.read_csv("../outputs/statusinvest.csv", sep=";")["TICKER"]
    statusinvest_tickers.columns = ['stock_ticker']

    # get deduped tickers list
    stock_tickers = pandas.concat([fundamentus_tickers, statusinvest_tickers]).drop_duplicates().values

In [4]:
@retry(stop=stop_after_attempt(5), wait=wait_random(min=1, max=5))
def get_stock_infos_df(stock_ticker: str) -> pandas.DataFrame:
    return StockInfosScraper(stock_ticker).dataframe_format

In [5]:
# get info from all stocks
stock_infos_df = None
failed_tickers = []

for stock_ticker in tqdm(stock_tickers):
    try:
        # read stock infos
        si_df = get_stock_infos_df(stock_ticker)
        si_df.columns = [stock_ticker]
        si_df = si_df.T
        stock_infos_df = pandas.concat([stock_infos_df, si_df])
    except:
        failed_tickers.append(stock_ticker)

# show company infos retrieved from Fundamentei
print(stock_infos_df[stock_infos_df.columns[0:4]])
print(f"Filed stock tickers: {failed_tickers}")


100%|██████████| 337/337 [08:04<00:00,  1.44s/it]

     Ticker       Nome da Empresa         Setor de Atuação Nota dos Usuários
PETR   PETR             Petrobras  Petróleo e Gás - Refino               3.8
ITUB   ITUB                  Itaú                   Bancos               4.9
VALE   VALE                  Vale                Mineração               4.3
ABEV   ABEV                 Ambev                  Bebidas               4.4
BPAC   BPAC           BTG Pactual                   Bancos               4.3
...     ...                   ...                      ...               ...
NEXP   NEXP                 Nexpe    Exploração de Imóveis                 1
MAPT   MAPT  Cemepe Investimentos                  Holding                 0
FRTA   FRTA           Pomi Frutas     Alimentos - Variados               1.7
PSVM   PSVM         Porto Sudeste    Logística e Portuário                 0
PPLA   PPLA                  PPLA                  Holding                 0

[336 rows x 4 columns]
Filed stock tickers: ['PMAM']


### Output result into a file   

In [6]:
stock_infos_df.to_csv("../outputs/fundamentei.csv", sep=";", index=False)